# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("data/sp_500_stocks.csv")


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [5]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [17]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'TR SASQKTAENEGLSCEM A)LO/LAG( BND', 'calculationPrice': 'close', 'open': None, 'openTime': None, 'openSource': 'flcfiaio', 'close': None, 'closeTime': None, 'closeSource': 'loffcaii', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 126.04, 'latestSource': 'Close', 'latestTime': 'December 4, 2020', 'latestUpdate': 1653070057365, 'latestVolume': None, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 123.81, 'previousVolume': 81489812, 'change': -0.71, 'changePercent': -0.00576, 'volume': None, 'iexMarketPercent': None, 'iexVolume': None, 'avgTotalVolume': 101938204, 'iexBidPrice': None, 'iexBidSize': Non

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [19]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price, market_cap)

126.04 2137019886194


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [22]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [28]:
final_dataframe.append(
    pd.Series([symbol, price, market_cap, 'N/A'],
              index=my_columns), 
    ignore_index=True)

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,126.04,2137019886194,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [35]:
final_datafram = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    
    final_dataframe = final_dataframe.append(
    pd.Series([stock, data['latestPrice'], data['marketCap'], 'N/A'],
              index=my_columns), 
    ignore_index=True)

In [36]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,118.420,35647373486,N/A
1,AAL,16.500,10040152273,N/A
2,AAP,153.590,10482810134,N/A
3,AAPL,122.730,2110794671871,N/A
4,ABBV,112.630,197675811038,N/A
...,...,...,...,...
505,YUM,109.600,32733916750,N/A
506,ZBH,153.520,31943734868,N/A
507,ZBRA,395.860,20468600171,N/A
508,ZION,46.179,7231152366,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [80]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [81]:
final_dataframe = pd.DataFrame()

In [82]:
symbol_groups = list(chunks(stocks['Ticker'], 100))

In [83]:
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))

In [84]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Market Capitalization,Number Of Shares to Buy,Price,Ticker
0,3.588826e+10,N/A,120.030,A
1,1.105669e+10,N/A,18.190,AAL
2,1.070995e+10,N/A,157.300,AAP
3,2.166805e+12,N/A,129.620,AAPL
4,1.868178e+11,N/A,106.090,ABBV
...,...,...,...,...
500,3.232194e+10,N/A,109.890,YUM
501,3.119344e+10,N/A,151.400,ZBH
502,2.092563e+10,N/A,386.636,ZBRA
503,7.401630e+09,N/A,44.200,ZION


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [85]:
portfolio_size = input("Enter the size of your portfolio size: ")

try: 
    portfolio_size = float(portfolio_size)
except:
    print("Please enter a floating number")

Enter the size of your portfolio size: 10000000


In [93]:
position_size = portfolio_size / len(final_dataframe.index)
for index, row in final_dataframe.iterrows():
    final_dataframe.loc[index, 'Number Of Shares to Buy'] = math.floor(position_size / row['Price'])

In [94]:
final_dataframe

,Market Capitalization,Number Of Shares to Buy,Price,Ticker
0,3.588826e+10,164,120.030,A
1,1.105669e+10,1088,18.190,AAL
2,1.070995e+10,125,157.300,AAP
3,2.166805e+12,152,129.620,AAPL
4,1.868178e+11,186,106.090,ABBV
...,...,...,...,...
500,3.232194e+10,180,109.890,YUM
501,3.119344e+10,130,151.400,ZBH
502,2.092563e+10,51,386.636,ZBRA
503,7.401630e+09,448,44.200,ZION


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [99]:
writer = pd.ExcelWriter('data/recommended_trades.xlsx', engine="xlsxwriter")
final_dataframe.to_excel(writer, "Recommended Trades", index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [100]:
background_color = "#0a0a23"
font_color = "#ffffff"

In [103]:
# All the excel format
string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [104]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [105]:
writer.save()

## A much faster to save it to CSV sheet

In [108]:
final_dataframe.to_csv("data/stocks.csv", index=False)